In [5]:
%reset

Exception ignored in: <function ZipFile.__del__ at 0x00000225F1EE5A60>
Traceback (most recent call last):
  File "C:\Users\ilija\AppData\Local\Programs\Python\Python38\lib\zipfile.py", line 1821, in __del__
    self.close()
  File "C:\Users\ilija\AppData\Local\Programs\Python\Python38\lib\zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


In [6]:
# import comet_ml
# from comet_ml import Experiment
# from comet_ml import OfflineExperiment
import tensorflow as tf
import importlib
import module_imports
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
import itertools
importlib.reload(module_imports)
from module_imports import *


Using tensorflow version: 2.4.0


In [7]:
INTERVAL_ID = 30
SCRIPT_ID = f'INST_{INTERVAL_ID}M_SDANN_SDNN_SD2_ZScoreByFeature'

# dirs
results_dir = module_paths.Path(f'Results_{module_time.ymd()}'); results_dir.create_dir(warn_exists = False)
cv_results_dir = module_paths.Path(f'CVResults_{module_time.ymd()}'); cv_results_dir.create_dir(warn_exists = False)
exception_dir = module_paths.Path(f'Exceptions'); exception_dir.create_dir(warn_exists = False)
metadata_dir = module_paths.Path('Metadata'); metadata_dir.create_dir(warn_exists = False)
metadata_folds_dir = module_paths.Path(f'Metadata\\Folds'); metadata_folds_dir.create_dir(warn_exists = False)
metadata_search_space_dir = module_paths.Path(f'Metadata\\SearchSpace'); metadata_search_space_dir.create_dir(warn_exists = False)
metadata_models_dir = module_paths.Path(f'ModelsData\\Models'); metadata_models_dir.create_dir(warn_exists = False)
trained_models_dir = module_paths.Path(f'TrainedModels'); trained_models_dir.create_dir()

# files
exceptions_file_path = f'{exception_dir.path}\\Exceptions_{SCRIPT_ID}.txt'

it # DATASET
HRV_FEATURES = ['SDNN','SDNN-1', 'ASDNN','ASDNN-1','ASDNN-2','ASDNN-3', 'SDANN','SDANN-1','SDANN-2','SDANN-3',
                'NN50','NN50-1','pNN50','pNN50-1', 'rMSSD','rMSSD-1','SD1','SD2','SD1/SD2']

WINDOWS_BEFORE = ['5-1', '1-0']
WINDOWS_AFTER = [f'{x}-{x-5}' for x in [15,10]] + ['5-1', '1-0']; WINDOWS_AFTER.reverse()
WINDOWS = [f'{w}-before' for w in WINDOWS_BEFORE] + [f'{w}-after' for w in WINDOWS_AFTER]


def preprocess_dataset(df: pd.DataFrame) -> pd.DataFrame:
    df = module_preprocessing.remove_outliers(df, features_to_analyze = set(df.columns) - {'Class', 'Glucose', 'Patient_ID'}, by_feature = True, method = 'z-score')
    df = module_preprocessing.power_transform(dataframe = df, features_to_ignore = {'Class', 'Patient_ID'})
    return df[['Patient_ID', 'Class'] + HRV_FEATURES].reset_index(drop = True)


dataset_all_windows = pd.read_excel(f'../data/Fasting_{INTERVAL_ID}min_instantaneous.xlsx')
dataset_all_windows = dataset_all_windows[dataset_all_windows['Class'] != 'ND']
dataset_all_windows.drop('Glucose', axis = 1, inplace = True)
dataset_all_windows.replace({'GD': 0, 'BD': 1}, inplace = True)

dataset = pd.DataFrame()
for window in WINDOWS:
    print(window)

    # select window, remove columns
    dataset_window = dataset_all_windows[(dataset_all_windows['Window'] == '-'.join(window.split('-')[:-1])) & (dataset_all_windows['BeforeAfter'] == window.split('-')[-1])].copy()
    dataset_window.drop(['Window', 'BeforeAfter'], axis = 1, inplace = True)

    # inplace preprocessing
    dataset_window = preprocess_dataset(dataset_window)
    dataset_window['Window'] = window
    dataset = pd.concat([dataset, dataset_window], sort = False).reset_index(drop = True)

dataset.replace(
        {
            '5-1-before': -5,
            '1-0-before': -1,
            '1-0-after': 1,
            '5-1-after': 5,
            '10-5-after': 10,
            '15-10-after': 15,
        },
        inplace = True
)

5-1-before
Number of rows removed:  0
1-0-before
Number of rows removed:  0
1-0-after
Number of rows removed:  0
5-1-after
Number of rows removed:  0
10-5-after
Number of rows removed:  0
15-10-after
Number of rows removed:  0


In [8]:
# FEATURE SELECTION
dataset = dataset[['Patient_ID', 'Class', 'SDANN', 'SDANN-1', 'SDNN', 'SDNN-1', 'SD2', 'Window']]
dataset.dropna(axis = 0, inplace = True)

# SEARCH SPACE
search_space = dict(
        batch_size = [1, 5],
        learning_rate = [0.2, 0.1, 0.02],
        optimizer = ['Nadam', 'Adam', 'RMSProp'],
        loss = ['CategoricalHinge', 'Poisson', 'CategoricalCrossentropy'],
        layers_list = [
            [
                {'units': 6, 'type': 'dense', 'activation': 'relu'},
                {'units': 3, 'type': 'dense', 'activation': 'relu'},
            ],
            [
                {'units': 6, 'type': 'dense', 'activation': 'relu'},
                {'units': 4, 'type': 'dense', 'activation': 'relu'},
                {'units': 2, 'type': 'dense', 'activation': 'relu'},
            ],
            [
                {'units': 6, 'type': 'dense', 'activation': 'relu'},
                {'units': 5, 'type': 'dense', 'activation': 'relu'},
                {'units': 4, 'type': 'dense', 'activation': 'relu'},
                {'units': 3, 'type': 'dense', 'activation': 'relu'},

            ],
            [
                {'units': 6, 'type': 'dense', 'activation': 'relu'},
                {'units': 5, 'type': 'dense', 'activation': 'relu'},
                {'units': 4, 'type': 'dense', 'activation': 'relu'},
                {'units': 3, 'type': 'dense', 'activation': 'relu'},
                {'units': 2, 'type': 'dense', 'activation': 'relu'},
            ],
        ],
        regularizer = ['tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)', None],
        initializer = ['VarianceScaling', 'GlorotNormal'],
        patience = [100],
        decay = [1e-2],
        early_stopping_flag = [True],
        validation_split = [0.05],
        epochs = [50,100,1000],
)

# combinations
keys, values = zip(*search_space.items())
combinations_dicts = [dict(zip(keys, v)) for v in itertools.product(*values)]

# tabular combinations
model_names = [f'M{i+1}' for i in range(len(combinations_dicts))]
combinations_df = pd.DataFrame(combinations_dicts)
combinations_df.insert(0, 'Model', model_names)
combinations_df['Trained'] = 'No'

# save
search_space_path = f'{metadata_search_space_dir.path}\\search_space_{SCRIPT_ID}.xlsx'
combinations_df.to_excel(search_space_path, index = False)

In [9]:
# FOLDS

random_states = [11,22,33,44,55,66,77,88,99,111]

split_sets_cv = dict()
train_folds_analysis_dict = dict()
test_folds_analysis_dict = dict()
train_folds_analysis_dict_overleaf = dict()
test_folds_analysis_dict_overleaf = dict()

for fold_id in range(1):

    fold_exists = True
    while fold_exists:

        train, test = module_train_test.find_best_group_split(dataframe = dataset,
                                                             target_feature = 'Class',
                                                             group_by_feature = 'Patient_ID',
                                                             balance_focus = 'train_with_test_threshold',
                                                             test_balance_treshold = 3,
                                                             num_splits_to_try = 1000,
                                                             random_state = random_states[fold_id],
                                                             test_size = 0.1)

        train_folds_analysis_dict[fold_id] = module_dataset_analysis.quantitative_analysis(df = train, dataset_name = f'fold{fold_id}',
                                                                                          class_feature = 'Class', classes = [0, 1])
        test_folds_analysis_dict[fold_id] = module_dataset_analysis.quantitative_analysis(df = test, dataset_name = f'fold{fold_id}',
                                                                                         class_feature = 'Class', classes = [0, 1])

        train_folds_analysis_dict_overleaf[fold_id] = module_dataset_analysis.quantitative_analysis(df = train, dataset_name = f'fold{fold_id}',
                                                                                                   class_feature = 'Class', classes = [0, 1],
                                                                                                   overleaf = True)
        test_folds_analysis_dict_overleaf[fold_id] = module_dataset_analysis.quantitative_analysis(df = test, dataset_name = f'fold{fold_id}',
                                                                                                  class_feature = 'Class', classes = [0, 1],
                                                                                                  overleaf = True)

        print(fold_id)
        if fold_id == 0:
            fold_exists = False
        else:
            for prev_fold_id in range(fold_id):
                print('Comparing with: \t\t\t\t',prev_fold_id)
                for test_analysis_key in test_folds_analysis_dict[fold_id].keys():
                    print('comparing', test_analysis_key)
                    if type(test_folds_analysis_dict[fold_id][test_analysis_key]) is str:
                        continue
                    if round(test_folds_analysis_dict[fold_id][test_analysis_key], 2) != round(test_folds_analysis_dict[prev_fold_id][test_analysis_key], 2):
                        print(f'{round(test_folds_analysis_dict[fold_id][test_analysis_key], 2)} != {round(test_folds_analysis_dict[prev_fold_id][test_analysis_key], 2)}')
                        fold_exists = False
                    else:
                        print('EQUAL VALUES')

        if fold_exists:
            random_states[fold_id] += 1
            print('_______________________________REPEATING SPLIT WITH DIFFERENT RANDOM SPLIT')

    train.reset_index(inplace = True, drop = True)
    test.reset_index(inplace = True, drop = True)

    train_patients = train.pop('Patient_ID')
    test_patients = test.pop('Patient_ID')

    overlap = list(set(train_patients.unique()).intersection(set(test_patients.unique())))
    print('Overlap: ' + str(overlap))
    assert dataset['Patient_ID'].nunique() == train_patients.nunique() + test_patients.nunique(), 'Problem wi`th patients'

    split_sets_cv[f'y_train_{fold_id}'] = train.pop('Class').to_numpy()
    split_sets_cv[f'y_test_{fold_id}'] = test.pop('Class').to_numpy()

    standard_scaler = MinMaxScaler()

    train = standard_scaler.fit_transform(train)
    test = standard_scaler.transform(test)

    split_sets_cv[f'X_train_{fold_id}'] = train
    split_sets_cv[f'X_test_{fold_id}'] = test

module_pandas_utils.print_df(pd.DataFrame(test_folds_analysis_dict).T)

pd.Series(random_states, name = 'random_states').to_excel(f'{metadata_folds_dir.path}\\random_states_{SCRIPT_ID}.xlsx', index = False)

pd.DataFrame(train_folds_analysis_dict).T.to_excel(f'{metadata_folds_dir.path}\\train_folds_analysis_{SCRIPT_ID}.xlsx', index = False)
pd.DataFrame(test_folds_analysis_dict).T.to_excel(f'{metadata_folds_dir.path}\\test_folds_analysis_{SCRIPT_ID}.xlsx', index = False)
pd.DataFrame(train_folds_analysis_dict_overleaf).T.to_excel(f'{metadata_folds_dir.path}\\train_folds_analysis_{SCRIPT_ID}_overleaf.xlsx', index = False)
pd.DataFrame(test_folds_analysis_dict_overleaf).T.to_excel(f'{metadata_folds_dir.path}\\test_folds_analysis_{SCRIPT_ID}_overleaf.xlsx', index = False)

# TRAINING

start_time = module_time.hmymd()
more_models_left_to_train = True

fold_results_list = list()
cv_results_list = list()

while more_models_left_to_train:

    search_space_state = pd.read_excel(search_space_path)

    model_to_train = search_space_state.loc[search_space_state['Trained'] == 'No'].iloc[0]
    print(f'Training model:\n {model_to_train}')

    search_space_state.loc[search_space_state['Model'] == model_to_train['Model'], 'Trained'] = 'InProgress'
    search_space_state.to_excel(search_space_path, index = False)

    model_fold_results_list = list()
    for fold_id in range(1):

        y_train = split_sets_cv[f'y_train_{fold_id}']
        y_test = split_sets_cv[f'y_test_{fold_id}']
        X_train = split_sets_cv[f'X_train_{fold_id}']
        X_test = split_sets_cv[f'X_test_{fold_id}']

        try:

            model_id = f"{model_to_train['Model']}_Fold{fold_id}"

            binary_classifier = module_dl.BinaryClassificationDL(X_train = X_train,
                                                                y_train = y_train,
                                                                X_test = X_test,
                                                                y_test = y_test,
                                                                epochs = model_to_train['epochs'],
                                                                batch_size = model_to_train['batch_size'],
                                                                learning_rate = model_to_train['learning_rate'],
                                                                decay = model_to_train['decay'],
                                                                validation_split = model_to_train['validation_split'],
                                                                loss_name = model_to_train['loss'],
                                                                initializer_name = model_to_train['initializer'],
                                                                optimizer_name = model_to_train['optimizer'],
                                                                early_stopping_flag = True,
                                                                patience = model_to_train['patience'],
                                                                layers_list = eval(model_to_train['layers_list']),
                                                                model_type = 'sequential',
                                                                regularizer = eval(model_to_train['regularizer']),
                                                                trained_models_dir = trained_models_dir,
                                                                model_id = model_id
                                                                )
            binary_classifier.fit_model()
            test_report = binary_classifier.predict(return_metrics = True)

            fold_results_list.append(dict(
                model = f"{model_to_train['Model']}_{fold_id}",
                **test_report,
                model_hyper = f"{model_to_train['Model']}_{fold_id}",
                **{key: value for (key, value) in model_to_train.to_dict().items() if key in search_space.keys() and len(search_space[key]) > 1},
                **{key: value for (key, value) in model_to_train.to_dict().items() if key in search_space.keys() and len(search_space[key]) == 1},
            ))
            model_fold_results_list.append(test_report)

        except Exception as e:

            print("ERROR: ",str(e))
            with open(exceptions_file_path, 'a') as exceptions_file:
                exceptions_file.write(f'{str(e)}\n')

    model_cv_results = pd.Series({'Model': model_to_train['Model']})\
        .append(pd.DataFrame(model_fold_results_list).mean())\
        .append(model_to_train).to_dict()

    cv_results_list.append(model_cv_results)

    pd.DataFrame(fold_results_list).to_excel(f'{results_dir.path}\\Fold_Results_{SCRIPT_ID}.xlsx')
    pd.DataFrame(cv_results_list).to_excel(f'{cv_results_dir.path}\\CV_Results_{SCRIPT_ID}.xlsx')

    search_space_state = pd.read_excel(search_space_path)
    search_space_state.loc[search_space_state['Model'] == model_to_train['Model'], 'Trained'] = 'Yes'
    search_space_state.to_excel(search_space_path, index = False)


0
Overlap: []
+----+--------------+------------------+-------------+-----------------+--------------+------------------+-------------+-----------------+----------------------+--------------------+------------+-----------+-----------+
|    |   0 patients |   0 patients (%) |   0 samples |   0 samples (%) |   1 patients |   1 patients (%) |   1 samples |   1 samples (%) |   Balance of samples |   Balance patients |   Patients |   Samples | dataset   |
|----+--------------+------------------+-------------+-----------------+--------------+------------------+-------------+-----------------+----------------------+--------------------+------------+-----------+-----------|
|  0 |            1 |           0.3333 |          22 |          0.2558 |            2 |           0.6667 |          64 |          0.7442 |               2.9091 |                  2 |          3 |        86 | fold0     |
+----+--------------+------------------+-------------+-----------------+--------------+------------------+

C:\Users\ilija\Pycharm Projects\GLYCO_open\modules\module_dl.py:568: UserWarning: Binary classifier ends with a layer with more than 2 units.Appending a sigmoid classification layer as a last layer of the neural network!
  warnings.warn('Binary classifier ends with a layer with more than 2 units.'


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 6)                 42        
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 21        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 4         
Total params: 67
Trainable params: 67
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/50
Epoch 2/50
Epoch 3/50
Epoch 4/50
Epoch 5/50
Epoch 6/50
Epoch 7/50
Epoch 8/50
Epoch 9/50
Epoch 10/50
Epoch 11/50
Epoch 12/50
Epoch 13/50
Epoch 14/50
Epoch 15/50
Epoch 16/50
Epoch 17/50
Epoch 18/50
Epoch 19/50
Epoch 20/50
Epoch 21/50
Epoch 22/50
Epoch 23/50
Epoch 24/50
Epoch 25/50
Epoch 26/50
Epoch 27/50
Epoch 28/50
Epoch 29/50
Epoch 30/50
Epoch 31/50
Epoch 32/50
Epoch 33

c:\users\ilija\pycharm projects\glyco_open\venv\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
C:\Users\ilija\Pycharm Projects\GLYCO_open\modules\module_dl.py:362: RuntimeWarning: invalid value encountered in longlong_scalars
  npv = round(true_negatives / (true_negatives + false_negatives), 4)
C:\Users\ilija\AppData\Local\Temp/ipykernel_9260/1686714756.py:156: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeErro

Training model:
 Model                                                                 M2
batch_size                                                             1
learning_rate                                                        0.2
optimizer                                                          Nadam
loss                                                    CategoricalHinge
layers_list            [{'units': 6, 'type': 'dense', 'activation': '...
regularizer                tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01)
initializer                                              VarianceScaling
patience                                                             100
decay                                                               0.01
early_stopping_flag                                                 True
validation_split                                                    0.05
epochs                                                               100
Trained                           

KeyboardInterrupt: 